<a href="https://colab.research.google.com/github/rochan17/Battery-Life-Prediction/blob/master/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet,LinearRegression
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error
from sklearn.model_selection import GridSearchCV,train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#making mean-percentage-error function
def avg_per_error(y_true,y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mean_error=make_scorer(avg_per_error,greater_is_better=False)

In [0]:
#loading Dataset
Dataset=pd.read_csv('/content/drive/My Drive/College/Research/Battery Ageing/final_dataset.csv')

In [0]:
Dataset_imp=Dataset[Dataset['cell_life']!=np.min(Dataset['cell_life'])]

In [0]:
numBat1=41
numBat2=43
numBat3=40
numBat=numBat1+numBat2+numBat3
test_ind = np.hstack((np.arange(0,(numBat1+numBat2),2),83))
train_ind = np.arange(1,(numBat1+numBat2-1),2)
secondary_test_ind = np.arange(numBat-numBat3,numBat)

Training_Data=Dataset.iloc[train_ind]
Testing_Data=Dataset.iloc[test_ind]
Secondary_Data=Dataset.iloc[secondary_test_ind]

In [0]:
#choosing model features 

varmod_features=['variance_dQ_100_10']

dismod_features=['minimum_dQ_100_10',
                 'variance_dQ_100_10',
                 'skewness_dQ_100_10',
                 'kurtosis_dQ_100_10',
                 'Discharge_Cap_2',
                 'Diff_MaxCap_DC2']

full_features=['minimum_dQ_100_10',
               'variance_dQ_100_10',
               'Slope_Cap_Fade_2_100',
               'Intercept_Cap_Fade_2_100',
               'Discharge_Cap_2',
               'Avg_charge_time',
               'Min_IR',
               'Diff_IR_2_100']

target_features=['cell_life']

In [0]:
#Model
var_params={'alpha':np.linspace(0.01,1,50),
       'l1_ratio':np.linspace(0.1,1,10)}

dis_params={'alpha':np.linspace(0.01,1,10),
       'l1_ratio':np.linspace(0.1,1,10)}

full_params={'alpha':np.linspace(0.1,1,10),
       'l1_ratio':np.linspace(0.1,1,10)}

def mod(features,target,params):
    
    X_train=Training_Data[features]
    y_train=Training_Data[target]
    X_test=Testing_Data[features]
    y_test=Testing_Data[target]
    X_sec=Secondary_Data[features]
    y_sec=Secondary_Data[target]
    
    RMSE=list()
    MPE=list()
    
    regression=ElasticNet(max_iter=50000,normalize=True)
    grid=GridSearchCV(estimator=regression,
                  param_grid=params,
                 cv=3,iid=False)
    grid.fit(X_train,y_train)
    
    print(grid.score(X_train,y_train))
    print(grid.best_params_)
    
    y_pred_train=grid.predict(X_train)
    y_pred_test=grid.predict(X_test)
    y_pred_sec=grid.predict(X_sec)
    
    RMSE.append(np.sqrt(mean_squared_error(y_train, y_pred_train)))
    MPE.append(float(np.mean(np.abs((y_train - y_pred_train.reshape(-1,1))) / y_train * 100)))
    
    RMSE.append(np.sqrt(mean_squared_error(y_test, y_pred_test)))
    MPE.append(float(np.mean(np.abs((y_test - y_pred_test.reshape(-1,1))) / y_test * 100)))

    RMSE.append(np.sqrt(mean_squared_error(y_sec, y_pred_sec)))
    MPE.append(float(np.mean(np.abs((y_sec - y_pred_sec.reshape(-1,1))) / y_sec * 100)))

    return RMSE,MPE

In [9]:
var_rmse,var_mpe=mod(varmod_features,target_features,var_params)

0.7796973328849709
{'alpha': 0.01, 'l1_ratio': 1.0}


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [10]:
dis_rmse,dis_mpe=mod(dismod_features,target_features,dis_params)

0.8734517260793405
{'alpha': 0.56, 'l1_ratio': 1.0}


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [11]:
full_rmse,full_mpe=mod(full_features,target_features,full_params)

0.9243295107561466
{'alpha': 0.6, 'l1_ratio': 1.0}


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:823: FutureWarning: The parameter 'iid' is deprecated in 0.22 and will be removed in 0.24.
  "removed in 0.24.", FutureWarning


In [12]:
pd.DataFrame({"Model":["Variance model", "Discharge model", "Full model"],
              "RMSE - Train": [var_rmse[0],dis_rmse[0],full_rmse[0]],
              "RMSE - Primary test": [var_rmse[1],dis_rmse[1],full_rmse[1]],
              "RMSE - Secondary test": [var_rmse[2],dis_rmse[2],full_rmse[2]],
              "MPE - Train": [var_mpe[0],dis_mpe[0],full_mpe[0]],
              "MPE - Primary test": [var_mpe[1],dis_mpe[1],full_mpe[1]],
              "MPE - Secondary test": [var_mpe[2],dis_mpe[2],full_mpe[2]]})                                                                       

,Model,RMSE - Train,RMSE - Primary test,RMSE - Secondary test,MPE - Train,MPE - Primary test,MPE - Secondary test
0,Variance model,151.665318,166.815772,183.028543,21.612829,22.229672,12.872008
1,Discharge model,114.948819,7016.056555,206.541474,16.021829,187.797192,14.724087
2,Full model,88.887292,172.062990,226.810949,11.131415,23.937359,15.836456
